In [20]:
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.tools import tool
from langchain.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

In [21]:
arquivo = "../data/dogma-e-ritual.pdf"

pure_Text = []
loader = PyPDFLoader(arquivo)
documento = loader.load()

In [22]:
for linha in documento:
    texto = linha.page_content
    texto_limpo = texto.replace("\n", " ").strip() 
    pure_Text.append(texto_limpo)
   

texto_completo = " ".join(pure_Text)
texto_final = texto_completo[6719:] 
#texto_final = texto_completo[6719:12000] 

print(texto_final)

Esoterismo Sacerdotal    Através do véu de todas as alegorias hieráticas e místicas dos antigos dogmas, através  das trevas e  provas bizarras de todas as iniciações, sob o selo de todas as escrituras sagradas, nas ruínas de  Nínive ou Tebas, sobre as pedras carcomidas dos antigos templos e a face escurecida das esfinges  da Assíria e do Egito, nas pinturas monstruosas  ou maravilhosas que produzem para o crente da  Índia e as páginas sagradas dos Vedas, nos emblemas estranhos dos nossos velhos livros de  alquimia, nas cerimônias de recepção praticadas por todas as sociedades misteriosas, encontram -se  os traços de uma dout rina em toda parte a mesma e em toda parte escondida cuidadosamente. A  filosofia oculta parece ter sido a nutriz ou matriz de todas as forças intelectuais, a chave de todas as  obscuridades divinas, e a rainha absoluta da sociedade, nos tempos em que era ex clusivamente  reservada à educação dos padres e dos reis.    Ela reinava na Pérsia com os magos, que um dia 

In [23]:
load_chunk = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 0,
    separators=[".", "!", "?", ";"]
)

documents = load_chunk.split_text(texto_final)

In [24]:
class TrainingData(BaseModel):
    human: str = Field(description="Uma pergunta que poderia gerar o texto de referência.")
    model: str = Field(description="Simular a resposta que Eliphas Levi daria com base no trecho de texto fornecido, mantendo sua linguagem, estilo e vocabulário.")


@tool(args_schema=TrainingData)
def parseamento(human: str, model: str):
    """Recebe um par 'human' e 'model' e o salva como uma nova linha
    em um arquivo JSONL. Esta ferramenta deve ser chamada para cada item gerado."""

    nome_do_arquivo = "eliphas_levi_dataset.jsonl"
    try:
        item_formatado = {"human": human, "model": model}
        
        with open(nome_do_arquivo, 'a', encoding='utf-8') as f:
            f.write(json.dumps(item_formatado, ensure_ascii=False) + '\n')
        return f"Item salvo com sucesso em {nome_do_arquivo}."
    
    except Exception as e:
        return f"Erro ao salvar o item: {e}"
    

llm = ChatOllama(
    model="qwen3:8b",
    temperature=0.4
) 


prompt = ChatPromptTemplate.from_messages([
            ("system", """
             
                - Você é um assistente especialista em criar datasets de alta qualidade para fine-tuning.
                - Sua tarefa é ler um trecho de texto do ocultista Eliphas Levi e criar uma pergunta relevante que um leigo/vulgo poderia fazer, e cuja resposta deve simular a resposta que Eliphas Levi daria com base no trecho de texto fornecido, mantendo sua linguagem e linha de pensamento a risca.
                - Considere substituir trechos abertos que mencionam apenas ciência ou filosofia oculta, por magia ou hermetismo (Observe que quando o usuário perguntar posteriormente, pode ser que ao falar de ciência queira falar de ciências físicas ao invés da magia). Observe e corrija coisas semelhantes, visando preparar um dataset para fine-tuning de um modelo de IA.

                <Não fazer>
                "Os presentes dos três magos — ouro, incenso e mirra — quais são suas conexões com os segredos da Cabala mencionados no texto?",
                "Qual é a natureza dessa força misteriosa que influenciava impérios e civilizações, segundo o texto?"
                "Como a força misteriosa mencionada no texto conseguiu influenciar impérios e civilizações antigas como a Grécia e a Índia?"
                ( Veja que nestes trechos acima, foi mencionado 'relacionado ao texto', 'mencionado no texto', 'segundo o texto' ou qualquer coisa do tipo. Estes termos não devem ser mencionados, pois somente você está vendo este texto, e como isso é um fine-tuning não faz sentido isso estar contido no datastorage).
                <Não fazer>

                CONTEXTO (Trecho do livro):
                {contexto}
            """),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])


tools = [parseamento]
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    max_iterations=1
)

print("\nIniciando a geração de dados sintéticos com o Agente...")
with open("eliphas_levi_dataset.jsonl", 'a', encoding='utf-8') as f:
    pass 

for i, chunk_text in enumerate(documents):
    print(f"\n--- Processando Chunk {i+1}/{len(documents)} ---")
    try:
       # print(f"------chunk_text======: {chunk_text}")
        resultado = agent_executor.invoke({"contexto": chunk_text})
        
    except Exception as e:
        print(f"Erro ao processar o chunk {i+1}: {e}")
        continue

print(f"\nGeração concluída! Verifique o arquivo 'eliphas_levi_dataset.jsonl'.")


Iniciando a geração de dados sintéticos com o Agente...

--- Processando Chunk 1/824 ---


> Entering new AgentExecutor chain...

Invoking: `parseamento` with `{'human': 'Como a filosofia oculta influenciou civilizações antigas como a Grécia e a Índia?', 'model': 'A filosofia oculta, como matriz de todas as forças intelectuais e chave das obscuridades divinas, moldou as civilizações antigas através da transmissão de segredos esotéricos. Sua influência se manifestou nas iniciações que preparavam os sacerdotes e reis, dotando-os da sabedoria para governar e interpretar os mistérios cósmicos. Nas ruínas dos templos e nas escrituras sagradas, restam traços dessa doutrina universal, que conecta a alma humana aos princípios herméticos da natureza, revelando que a verdadeira ciência é a arte de compreender a alquimia das realidades ocultas.'}`
responded: <think>
Okay, let's tackle this query. The user wants me to act as an expert in creating high-quality datasets for fine-tuning. My task is 